In [0]:
#To install the PyTorch 0.4
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

In [0]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torchvision import transforms, datasets

In [0]:
import numpy as np

def one_hot_transform(labels):
    one_hot = torch.zeros(labels.size()+tuple([10])).scatter_(-1, labels, 1)
    return one_hot 
  
trainset = datasets.MNIST ( root='../data', train = True, download = True, transform = transforms.Compose([ transforms.Pad(2), 
                                                                                                            transforms.RandomCrop(28),
                                                                                                            transforms.ToTensor() ]),
                            target_transform = one_hot_transform)

testset = datasets.MNIST ( root='../data', train = False, download = True, transform = transforms.Compose([ transforms.ToTensor() ]), target_transform = one_hot_transform)


train_loader = torch.utils.data.DataLoader( trainset, batch_size = 32, shuffle = True, num_workers = 4)

test_loader = torch.utils.data.DataLoader( testset, batch_size = 32, shuffle = False, num_workers = 4)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
class CapsLayer(nn.Module):
    def __init__(self, type_="primary", in_channels=32, out_channels=32, num_iterations=3, num_capsules=1, kernel_size=3, stride=2):
        super(CapsLayer, self).__init__()
        self.type_=type_
        self.in_channels=in_channels
        self.out_channels=out_channels
        if type_=="primary":
            self.kernel_size=kernel_size
            self.stride=stride
            self.conv=nn.Conv2d(in_channels, out_channels*17, 
                        kernel_size=1, stride=1, padding=0)
        else:
            self.num_iterations=num_iterations
            self.kernel=torch.eye(kernel_size, kernel_size)
            self.kernel=(self.kernel[:, None, :, None]*self.kernel[None, :, None, :]).view(1, -1, kernel_size, kernel_size)
            self.kernel=self.kernel.expand(in_channels*17, -1, kernel_size, kernel_size)\
                        .contiguous().view(-1, 1, kernel_size, kernel_size).to(device)
            self.in_channels=in_channels
            self.out_channels=out_channels
            self.kernel_size=kernel_size
            self.route_weights = nn.Parameter(torch.randn(out_channels, in_channels, 4, 4)).to(device)
            self.Ba=nn.Parameter(torch.randn(out_channels)).to(device)
            self.Bu=nn.Parameter(torch.randn(out_channels)).to(device)
            self.stride=stride
            #self.kernel=torch.ones(in_channels*17, 1, kernel_size, kernel_size).to(device)
                
            
            
            
    def forward(self, x):
        if self.type_=="primary":
            M=self.conv(x)
            
        elif self.type_=="conv":
            x=F.conv2d(x, self.kernel, groups=self.in_channels*17,  stride=self.stride)
            x=x.view(-1, self.in_channels*17, self.kernel_size**2, x.size(2), x.size(3))
            k_square=x.size(2)
            w=x.size(3)
            num_caps_j=self.out_channels
            num_caps_i=self.in_channels*k_square
            
            pose_i=x[:, :-self.in_channels, :, :, :].view(-1, self.in_channels, 4, 
                                                       4, x.size(2), x.size(3), x.size(4))\
                      .permute(0, 1, 4, 5, 6, 2, 3).contiguous()
            #pose_i=x[:, :-self.in_channels, :, :].view(-1, self.in_channels, 4, 
            #                                           4, x.size(2), x.size(3))\
            #          .permute(0, 1, 4, 5, 2, 3).contiguous()
            Ai=x[:, -self.in_channels:, :, :, :].view(-1, 1, num_caps_i, w, w, 1)
            #Ai=x[:, -self.in_channels:, :, :].view(-1, 1, num_caps_i, w, w, 1)
            
            V=(pose_i[:, None, :, :, :, :, :, :] @ self.route_weights[None, :, :, None, None, None, :, :])\
                      .view(-1, num_caps_j, num_caps_i, w, w, 16)
            #V=(pose_i[:, None, :, :, :, :, :] @ self.route_weights[None, :, :, None, None, :, :])\
            #          .view(-1, num_caps_j, num_caps_i, w, w, 16)
            
            R=torch.ones(V.size(0), V.size(1), V.size(2), V.size(3), V.size(4), 1).to(device)/num_caps_j
            for t in range(self.num_iterations):
                # M-step
                R=R*Ai
                M=(R*V).sum(dim=2,keepdim=True)/R.sum(dim=2,keepdim=True)
                E=(R*(M-V)**2).sum(dim=2,keepdim=True)/R.sum(dim=2,keepdim=True)
                cost=(self.Bu[None, :, None, None, None, None]+torch.log(M))*R.sum(dim=2,keepdim=True)
                Aj=F.sigmoid((t+1)*(self.Ba[None, :, None, None, None, None]-cost.sum(dim=-1,keepdim=True)))
                
                # E-step
                P=(-((V-M)**2/(2*E)).sum(dim=3,keepdim=True)).exp()/(2*np.pi*E).prod(dim=-1,keepdim=True)
                R=Aj*P/(Aj*P).sum(dim=1,keepdim=True)
            M=M.view(-1, self.out_channels, w, w, 4, 4)
            M=M.view(-1, self.out_channels, w, w, 4, 4).permute(0, 1, 4, 5, 2, 3).contiguous().view(-1, 16*self.out_channels, w,w)
            Aj=Aj.view(-1, self.out_channels, w, w)
                
            M=torch.cat([M, Aj], dim=1)
            
        else:
            w=x.size(2)
            num_caps_j=self.out_channels
            num_caps_i=self.in_channels*w*w #*k_square
            
            pose_i=x[:, :-self.in_channels, :, :].view(-1, self.in_channels, 4, 
                                                       4, x.size(2), x.size(3))\
                      .permute(0, 1, 4, 5, 2, 3).contiguous()
            Ai=x[:, -self.in_channels:, :, :].view(-1, 1, num_caps_i, 1)
            
            V=(pose_i[:, None, :, :, :, :, :] @ self.route_weights[None, :, :, None, None, :, :])\
                      .view(-1, num_caps_j, num_caps_i, 16)
            
            R=torch.ones(V.size(0), V.size(1), V.size(2), 1).to(device)/num_caps_j
            for t in range(self.num_iterations):
                # M-step
                R=R*Ai
                M=(R*V).sum(dim=2,keepdim=True)/R.sum(dim=2,keepdim=True)
                E=(R*(M-V)**2).sum(dim=2,keepdim=True)/R.sum(dim=2,keepdim=True)
                cost=(self.Bu[None, :, None, None]+torch.log(M))*R.sum(dim=2,keepdim=True)
                Aj=F.sigmoid((t+1)*(self.Ba[None, :, None, None]-cost.sum(dim=-1,keepdim=True)))
                
                # E-step
                P=(-((V-M)**2/(2*E)).sum(dim=3,keepdim=True)).exp()/(2*np.pi*E).prod(dim=-1,keepdim=True)
                R=Aj*P/(Aj*P).sum(dim=1,keepdim=True)
            M=M.view(-1, self.out_channels, 4, 4)
            M=M.view(-1, self.out_channels, 4, 4).view(-1, 16*self.out_channels)
            Aj=Aj.view(-1, self.out_channels)
                
            M=torch.cat([M, Aj], dim=1)
        return M

In [0]:
class CapsNet(nn.Module):
    def __init__(self):
        super(CapsNet, self).__init__()
        
        self.conv1=nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, 
                             stride=2, padding=2)
        
        self.primary=CapsLayer(type_="primary", in_channels=32, out_channels=32,
                               kernel_size=1, stride=1)
        
        self.convCaps1=CapsLayer(type_="conv", in_channels=32, out_channels=32, 
                               kernel_size=3, stride=4)
        
        self.convCaps2=CapsLayer(type_="conv", in_channels=32, out_channels=32, 
                               kernel_size=3, stride=1)
        
        self.classCaps=CapsLayer(type_="class", in_channels=32, out_channels=10)
        
        self.reconst=nn.Sequential(
            nn.Linear(17*10 ,512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 784),
            nn.Sigmoid()
        )
    
    def reconstruction(self, x, y):
        recon_images = self.reconst((x*y[:,:,None]).view(-1, 170)).view(-1, 28, 28)
        return recon_images

    def forward(self, x, y):
        l1=F.relu(self.conv1(x))
        l2=self.primary(l1)
        l3=self.convCaps1(l2)
        l4=self.convCaps2(l3)
        l5=self.classCaps(l4).view(-1, 10, 17)
        
        preds=l5[:, :, -1].view(-1, 10)
        
        return preds, self.reconstruction(l5, y)

In [0]:
class CapsLoss(nn.Module):
    def __init__(self):
        super(CapsLoss, self).__init__()
        
    def forward(self, images, labels, preds, reconst_images):
        preds=torch.clamp(preds, min=0.1, max=0.9)
        margin_loss=(labels*((0.9-preds)**2) + 0.5*(1-labels)*((preds-0.1)**2)).sum()
        
        reconst_loss=((images-reconst_images)**2).sum()
        return (margin_loss+0.000*reconst_loss)/images.size(0)

In [0]:
import torch.optim as optim

criterion = CapsLoss()
net=CapsNet().to(device) #for GPU
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    count=0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs, reconst = net(inputs, labels)
        loss = criterion(inputs, labels, outputs, reconst)
        
        L=labels.cpu().detach().numpy()
        P=outputs.cpu().detach().numpy()
        P=P.argmax(axis=-1)
        L=L.argmax(axis=-1)
        count+=np.array([L[it]==P[it] for it in range(L.shape[0])]).sum()
        
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f count: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100, count / 100))
            running_loss = 0.0
            count = 0.0